In [4]:
import xml.etree.ElementTree as ET
import json

def get_full_tag(element):
    """
    Helper function to retrieve the full tag without namespace.
    """
    return element.tag.split('}')[-1] if '}' in element.tag else element.tag

def parse_connection(connection_element):
    """
    Parses a <connection> element and returns a dictionary of its attributes.
    """
    connection = {}
    for attr in connection_element.attrib:
        connection[attr] = connection_element.get(attr)
    return connection

def parse_relation(relation_element):
    """
    Parses a <relation> element and returns a dictionary of its attributes.
    """
    relation = {}
    for attr in relation_element.attrib:
        relation[attr] = relation_element.get(attr)
    return relation

def parse_metadata_record(metadata_element):
    """
    Parses a <metadata-record> element and returns a dictionary of its child elements.
    """
    record = {}
    for child in metadata_element:
        tag = get_full_tag(child)
        record[tag] = child.text
    return record

def parse_calculation(calculation_element):
    """
    Parses a <calculation> element and returns a dictionary of its attributes and formula.
    """
    calculation = {
        "class": calculation_element.get('class'),
        "formula": calculation_element.get('formula')
    }
    return calculation

def parse_datasources(root):
    """
    Parses all datasources from the XML root and extracts required fields.

    Expected JSON Structure for each datasource:
    {
        "name": "named_connection_name",
        "caption": "datasource_caption",
        "connection": {
            "named_connection_name": {
                "type": "connection_class",
                "server": "server_name",
                "database": "database_name",
                "port": "port_number",
                "authentication": "authentication_method",
                "class": "connection_class"
            },
            // More named connections if present
        },
        "relations": [
            {
                "name": "relation_name",
                "connection": "relation_connection",
                "table": "table_name",
                "type": "relation_type"
            },
            // More relations
        ],
        "columns": [
            {
                "key": "column_key",
                "value": "column_value"
            },
            // More columns
        ],
        "metadata_records": [
            {
                "remote-name": "remote_column_name",
                "remote-type": "type_code",
                "local-name": "local_column_name",
                "parent-name": "parent_table",
                // Additional metadata attributes
            },
            // More metadata records
        ],
        "calculations": [
            {
                "class": "calculation_class",
                "formula": "calculation_formula"
            },
            // More calculations
        ]
    },
    // More datasources
    """
    datasources = []

    # Iterate through each <datasource> element
    for ds in root.findall('.//datasources/datasource'):
        datasource = {
            "name": ds.get('name') or None,
            "caption": ds.get('caption') or None,
            "connection": {},
            "relations": [],
            "columns": [],
            "metadata_records": [],
            "calculations": []
        }

        # Parse named-connections
        named_connections = ds.find('connection/named-connections')
        if named_connections is not None:
            for nc in named_connections.findall('named-connection'):
                connection = nc.find('connection')
                if connection is not None:
                    # Safely extract each required attribute, assigning None if absent
                    conn_info = {
                        "type": connection.get('class') or None,
                        "server": connection.get('server') or None,
                        "database": connection.get('dbname') or None,
                        "port": connection.get('port') or None,
                        "authentication": connection.get('authentication') or None,
                        "class": connection.get('class') or None
                    }
                    # Use the named connection's name as the key
                    conn_name = nc.get('name') or "default"
                    datasource["connection"][conn_name] = conn_info
                else:
                    print(f"Warning: <connection> not found within <named-connection> '{nc.get('name')}'. Skipping.")
        else:
            print(f"Warning: <named-connections> not found within <datasource> '{ds.get('name')}'. Skipping.")

        # Parse all <relation> elements within the datasource
        for rel in ds.iter():
            if get_full_tag(rel) == 'relation':
                relation = parse_relation(rel)
                if any(relation.values()):  # Ensure that there's at least one non-null value
                    datasource["relations"].append(relation)

        # Parse <cols>/<map> elements
        cols = ds.find('cols')
        if cols is not None:
            for col_map in cols.findall('map'):
                column = {
                    "key": col_map.get('key') or None,
                    "value": col_map.get('value') or None
                }
                datasource["columns"].append(column)

        # Parse <metadata-records>/<metadata-record> elements
        meta = ds.find('metadata-records')
        if meta is not None:
            for m in meta.findall('metadata-record'):
                record = parse_metadata_record(m)
                # Only append if there's meaningful data
                if any(record.values()):
                    datasource["metadata_records"].append(record)

        # Parse calculations within <column> elements
        for column in ds.findall('column'):
            calc = column.find('calculation')
            if calc is not None:
                calculation = parse_calculation(calc)
                if any(calculation.values()):
                    datasource["calculations"].append(calculation)

        # Append the fully populated datasource to the list
        datasources.append(datasource)

    return datasources


In [5]:
def parse_worksheets(root):
    """Parse all worksheets from the XML root."""
    worksheets = []
    for ws in root.findall('.//worksheets/worksheet'):
        worksheet = {
            "name": ws.get('name'),
            "datasources": [],
            "filters": [],
            "slices": [],
            "aggregation": None
            # Add other worksheet attributes as needed
        }

        table = ws.find('table')
        if table is not None:
            view = table.find('view')
            if view is not None:
                # Parse datasources used in the worksheet
                for ds in view.findall('.//datasources/datasource'):
                    ds_name = ds.get('name')
                    if ds_name:
                        worksheet["datasources"].append(ds_name)

                # Parse filters
                for filt in view.findall('.//filter'):
                    filter_item = {
                        "class": filt.get('class'),
                        "column": filt.get('column'),
                        "function": filt.find('.//groupfilter').get('function') if filt.find('.//groupfilter') is not None else None
                        # Add more filter attributes as needed
                    }
                    worksheet["filters"].append(filter_item)

                # Parse slices
                for sl in view.findall('.//slices/column'):
                    if sl.text:
                        worksheet["slices"].append(sl.text)

                # Parse aggregation
                agg = view.find('aggregation')
                if agg is not None:
                    worksheet["aggregation"] = agg.get('value')

        worksheets.append(worksheet)
    return worksheets

def parse_dashboards(root):
    """Parse all dashboards from the XML root."""
    dashboards = []
    for db in root.findall('.//dashboards/dashboard'):
        dashboard = {
            "name": db.get('name'),
            "size": {},
            "datasources": [],
            "zones": []
            # Add other dashboard attributes as needed
        }

        # Parse size
        size = db.find('size')
        if size is not None:
            dashboard["size"] = {
                "minwidth": size.get('minwidth'),
                "minheight": size.get('minheight'),
                "maxwidth": size.get('maxwidth'),
                "maxheight": size.get('maxheight'),
                "sizing_mode": size.get('sizing-mode')
            }

        # Parse datasources used in the dashboard
        for ds in db.findall('.//datasources/datasource'):
            ds_name = ds.get('name')
            if ds_name:
                dashboard["datasources"].append(ds_name)

        # Parse zones (layout details)
        for zone in db.findall('.//zones/zone'):
            zone_details = {
                "id": zone.get('id'),
                "type": zone.get('type-v2'),
                "name": zone.get('name'),
                "x": zone.get('x'),
                "y": zone.get('y'),
                "w": zone.get('w'),
                "h": zone.get('h')
                # Add more zone attributes as needed
            }
            dashboard["zones"].append(zone_details)

        dashboards.append(dashboard)
    return dashboards

def parse_actions(root):
    """Parse all actions from the XML root."""
    actions = []
    for action in root.findall('.//actions/action'):
        act = {
            "name": action.get('name'),
            "caption": action.get('caption'),
            "activation": {},
            "source": {},
            "command": {}
        }

        # Parse activation
        activation = action.find('activation')
        if activation is not None:
            act["activation"] = {
                "auto_clear": activation.get('auto-clear'),
                "type": activation.get('type')
            }

        # Parse source
        source = action.find('source')
        if source is not None:
            act["source"] = {
                "dashboard": source.get('dashboard'),
                "type": source.get('type'),
                "worksheet": source.get('worksheet')
            }

        # Parse command
        command = action.find('command')
        if command is not None:
            cmd = {
                "command": command.get('command'),
                "params": {}
            }
            for param in command.findall('param'):
                cmd["params"][param.get('name')] = param.get('value')
            act["command"] = cmd

        actions.append(act)
    return actions

def main():
    # Replace 'sample.xml' with the path to your Tableau XML workbook
    xml_file = 'Unlicensed Users Owning Objects S-R2.xml'
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Parse different entities
    datasources = parse_datasources(root)
    worksheets = parse_worksheets(root)
    dashboards = parse_dashboards(root)
    actions = parse_actions(root)

    # Compile everything into a single dictionary
    workbook_json = {
        "datasources": datasources,
        "worksheets": worksheets,
        "dashboards": dashboards,
        "actions": actions
    }

    # Output to a JSON file
    with open(xml_file + '.json', 'w', encoding='utf-8') as f:
        json.dump(workbook_json, f, indent=4, ensure_ascii=False)

    print("JSON output has been written to 'workbook.json'.")

if __name__ == "__main__":
    main()


JSON output has been written to 'workbook.json'.
